In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("census_data_new.csv")
df.head()

In [ ]:
df.info()

In [ ]:
df = df.dropna(how='all', axis=1)
df.info()

In [ ]:
df = df.dropna(thresh=100, axis = 1)
df.info()

In [ ]:
df = df.sort_index(axis = 1)

In [ ]:
df = df.convert_dtypes()

In [ ]:
df = df.replace(to_replace='Mifugo yote/ Bidhaa zote za mifugo', value='Mifugo yote')

In [ ]:
df = df.replace(to_replace='Agricultural porocessing', value='Agricultural processing')

In [ ]:
df = df.replace(to_replace='Kibanga Agro Milling Co Ltd Kibanga Agro Milling Co Ltd', value='Kibanga Agro Milling Co Ltd')

In [ ]:
df['sector'] = df['sector'].replace(to_replace="Skilled Trades", value="Repairs")

In [ ]:
df.to_csv("census_data_trimmed.csv", index=False)

In [ ]:
df.columns

In [ ]:
pd.unique(df[['specialty_swa_1', 'specialty_swa_2',
       'specialty_swa_3', 'specialty_swa_4']].values.ravel('K'))

In [ ]:
pd.unique(df[['livestock_swa1']].values.ravel('K'))

In [ ]:
pd.unique(df[['crop_swa1', 'crop_swa2', 'crop_swa3']].values.ravel('K'))

In [ ]:
pd.unique(df[['inputs_swa1', 'inputs_swa2', 'inputs_swa4']].values.ravel('K'))

In [ ]:
pd.unique(df[['subsector_swa_1','subsector_swa_2']].values.ravel('K'))

In [ ]:
[col for col in pd.unique(df[['subsector_eng_1','subsector_eng_2']].values.ravel('K')) if not pd.isna(col) and col in [c for c in pd.unique(df[['subsector_swa_1','subsector_swa_2']].values.ravel('K')) if not pd.isna(c)]]

In [ ]:
pd.unique(df[['sector']].values.ravel('K'))

In [ ]:
pd.unique(df.loc[df['sector'] == 'Agricultural processing'][['subsector_swa_1','subsector_swa_2']].values.ravel('K'))

In [ ]:
len(pd.unique(df[['subsector_swa_1','subsector_swa_2', 'specialty_swa_1', 'specialty_swa_2',
       'specialty_swa_3', 'specialty_swa_4']].values.ravel('K')))

In [ ]:
s = 0
for row in df['firm_name']:
    s = max(s,len(row))
print(s)

In [ ]:
sorted([val for val in pd.unique(df[df['sector'] == 'Services'][['subsector_swa_1','subsector_swa_2']].values.ravel('K')) if not pd.isnull(val)])

In [ ]:
pd.unique(df['sector'])

In [ ]:
len([name for i, name in enumerate(df['firm_name']) if name in df['ownername'][i]])

In [ ]:
df = pd.read_csv('firm_visibility_assignments.csv')

In [ ]:
df.head()

In [ ]:
df[~df['mobile_number2'].isna()]

In [ ]:
numberGroups = dict()
for num1, num2 in zip(df[~df['mobile_number2'].isna()]['mobile_number1'], df[~df['mobile_number2'].isna()]['mobile_number2']):
    if num1 not in numberGroups:
        numberGroups[num1] = set()
    if num2 not in numberGroups:
        numberGroups[num2] = set()
    numberGroups[num1].add(num1)
    numberGroups[num1].add(num2)
    for n in numberGroups[num1]:
        if n not in numberGroups:
            numberGroups[n] = set()
        if n == num1:
            continue
        numberGroups[n].add(num2)
        numberGroups[n].add(num1)
    numberGroups[num2].add(num2)
    numberGroups[num2].add(num1)
    for n in numberGroups[num2]:
        if n not in numberGroups:
            numberGroups[n] = set()
        if n == num2:
            continue
        numberGroups[n].add(num2)
        numberGroups[n].add(num1)
print(len(numberGroups))

In [ ]:
for group in numberGroups.values():
    if len(group) > 2:
        print(group)

In [ ]:
numbers = pd.concat([df['mobile_number1'], df['mobile_number2'].dropna()], ignore_index=True).astype('int64').unique()

In [ ]:
rows = []
for number in numbers:
    group = 'A' if list(df[(df['mobile_number1'] == number) | (df['mobile_number2'] == number)]['owner_names_visible'])[0] == 1 else 'B'
    if number in numberGroups:
        twoNumbers = sorted(list(numberGroups[number]))
        rows += [[twoNumbers[0], twoNumbers[1], twoNumbers[-1] if len(twoNumbers) > 2 else pd.NA, group]]
    else:
        rows += [[number, pd.NA, pd.NA, group]]

In [ ]:
df3 = pd.DataFrame(rows, columns=['phone_num_1','phone_num_2','phone_number_3','group'])

In [ ]:
df3.head(20)

In [ ]:
df3 = df3.sample(frac=1).reset_index(drop=True)

In [ ]:
df3.drop_duplicates(subset='phone_num_1', inplace=True)

In [ ]:
df3.groupby('group').count()

In [ ]:
df4 = pd.read_csv('census_data_trimmed.csv')

In [ ]:
business = pd.concat([df4['mobile_number1'], df4['mobile_number2']], ignore_index=True).dropna().astype('int64').unique()

In [ ]:
[number for number in business if number not in df3['phone_num_1'].dropna().astype('int64').unique() and number not in df3['phone_num_2'].dropna().astype('int64').unique() and number not in df3['phone_number_3'].dropna().astype('int64').unique()]

In [ ]:
[number for number in pd.concat([df3['phone_num_1'], df3['phone_num_2'], df3['phone_number_3']], ignore_index=True).dropna().astype('int64').unique() if number not in business]

In [ ]:
df3.to_csv('AB_test_groups.csv', index=False)

In [ ]:
for key, group in numberGroups.items():
    if len(group) > 2:
        print(group)

In [ ]:
df.groupby('owner_names_visible').nunique()

In [ ]:
df2 = pd.read_csv('AB_test_groups.csv')

In [ ]:
sum(df2[df2.duplicated(subset='phone_num_1')]['phone_num_2'].isna())

In [ ]:
df2 = df2.sample(frac=1).reset_index(drop=True)

In [ ]:
df2.drop_duplicates(subset='phone_num_1', inplace=True)

In [ ]:
df2.head()

In [ ]:
df2.groupby('group').nunique()

In [ ]:
from scipy.stats import chisquare

In [ ]:
chisquare([4594, 4620])

In [ ]:
df2.to_csv('AB_test_groups.csv', index=False)